# Luminosity at the FCC-ee Collider - Data Analysis

The steps needed to make your comparison of Standard Model predictions to **simulated** data for Bhabha scattering are described in this notebook.

The data we will examine come from a simulation of an experimental analysis for the process $e^+e^- \rightarrow e^+ e^-$. We aim to study the distribution as a function of scattering angle:
$$\frac{\mathrm{d}\sigma}{\mathrm{d}\theta},$$
which is measured in units of $\mathrm{pb}$. This will plotted in bins of $\theta$, measured in radians.

In this notebook you will use small angle Bhabha scattering to determine the integrated luminosity of the FCC-ee, by comparing experimental data on the number of events, to a theoretical prediction for the cross-section.

___
## Structuring data

Information can be stored in many form on computing systems, **data** needs to be **structured** in sensible and comprehensible ways that simplify interaction and interpreting.

A useful data structure in python (which exists in other programming languages) is the **array**, part of the `numpy` library.

Arrays are **containers** of variables which can be accessed individually, looped over, or altered by functions. Examples of how to use them are shown below - experiment with different values and functionalities!

In [ ]:
## Example 1: Simple array manipulations
import numpy as np

# Construct arrays of numbers
array_1 = np.array([1,2,3,4])
array_2 = np.array([5,6,7,8])

print("Printing arrays")
print(array_1)
print(array_2)

# Operations for arrays
print("\nSimple array operations")
print("array_1 * array_2 = ", array_1 * array_2) # multiplies each element of array_1 by the corresponding element of array_2
print("array_1 + array_2 = ", array_1 + array_2) # adds each element of array_1 to the corresponding element of array_2
print("array_1 - array_2 = ", array_1 - array_2) # subtracts from each element of array_1 the corresponding element of array_2
print("array_1 / array_2 = ", array_1 / array_2) # divides each element of array_1 by the corresponding element of array_2

# Indexing for access to specific elements
# The first element of the array is '0', the second is '1', ...
print("\nIndexing arrays")
print("array_1[0] = ", array_1[0])
print("array_1[1] = ", array_1[1])
print("array_1[2] = ", array_1[2])
print("array_1[3] = ", array_1[3])

Arrays can also be **multi-dimensional**!

In [ ]:
## Example 2: Multi-dimensional arrays
import numpy as np

multi_array_1 = np.array([[1,2],[3,4]]) # This forms a 2x2 matrix!
multi_array_2 = np.array([[5,6],[7,8]])

# The 'shape' function tells you the dimensions of the array
print("Shape of multi_array_1 = ", np.shape(multi_array_1)) # 2x2 matrix
print("Shape of multi_array_2 = ", np.shape(multi_array_2)) # 2x2 matrix

# Multi-dimensional array operations:
print("\n\nMulti-dimensional array operations")
print("multi_array_1 = \n", multi_array_1)
print("multi_array_2 = \n", multi_array_2)
print("multi_array_1 + multi_array_2 = \n", multi_array_1 + multi_array_2)

# Indexing is more complicated:
print("\n\nIndexing multi-dimensional arrays")
print("multi_array_1[0,0] = ", multi_array_1[0,0])
print("multi_array_1[0,1] = ", multi_array_1[0,1])
print("multi_array_1[1,0] = ", multi_array_1[1,0])
print("multi_array_1[1,1] = ", multi_array_1[1,1])

You can also access a whole row or column at the same time - this will come in useful later when plotting, as most `matplotlib` functions will accept an array to plot multiple data points all at once.
You can also set whole columns at once!

In [ ]:
# We access an entire row or column by using a colon
print("Accessing entire rows or columns")
print("multi_array_1[0,:] = ", multi_array_1[0,:]) # First row
print("multi_array_1[:,0] = ", multi_array_1[:,0]) # First column

multi_array_1[:,1] = np.array([9,10]) # We can set an entire column at once
print("multi_array_1 after setting second column to [9,10]: \n", multi_array_1)

___

## Reading external files

Data from experiments and from theoretical predictions are often stored in external files, which can be imported into arrays.

We have provided data in the `.dat` files in the same folder as this notebook.
The experimental data is provided as a list of data points in the following format:
$$[\mathtt{x\_low \quad x\_high \quad value \quad error}],$$
where $\mathtt{x\_low,\ x\_high}$ are the edges of the bins in $x$, $\mathtt{value}$ is the value of the histogram at that point, and $\mathtt{error}$ is the error of $y$.

We can read these in python in a number of different ways:

In [ ]:
import numpy as np

# Native python open() function
# 'r' argument means 'read-only'
with open("small-angle-data.dat", "r") as f:
    print(f.read())
    print(type(f.read()),"\n\n") # The native python 'open' function converts the file read to a string - not a useful format

# Try with numpy - which has some smarter functions!
experimental_data = np.genfromtxt("small-angle-data.dat", dtype=float)
print(experimental_data)
print(type(experimental_data)) # The method has converted the data to a numpy array!
print(np.shape(experimental_data)) # The dimensions are right too - 5 bins, 4 columns for each bin!

# We can print all the measured values at once like we learned above
print(experimental_data[:,2]) # Print third column

___
#### Exercise 5a: Making theoretical predictions

Now we will calculate the theory prediction for the cross-section, as integrated over each the experimental angular bin.

Below is a pre-written version of the Monte-Carlo functions you have previously written, and the SM cross-section function.

In [ ]:
import random
import numpy as np

def integrate_myfunction_a_b(n_points, my_function, a, b):
    V = (b - a)
    sum = 0
    for i in range(n_points):
        # Generate a random number between a and b
        x = random.uniform(a, b)
        # Calculate the value of the function at x
        f_x = my_function(x)
        # Add the value to the sum
        sum += f_x
    # Your code here
    integral = (V * sum) / n_points
    return integral

def final_monte_carlo_function(my_function, a, b):
    # Does Monte-Carlo integration of `my_function` over the interval [a, b] with 10,000 random samples,
    # 10 times and returns the mean and standard deviation of the results.

    values = [integrate_myfunction_a_b(10000, my_function, a, b) for _ in range(10)]
    # Then calculate the mean and standard deviation of the results
    mean = sum(values)/10 # Calculate the mean of the results
    std_dev = (sum((x - mean)**2 for x in values) / 10)**0.5 # Calculate the standard deviation of the results
    return (mean, std_dev)

In [ ]:
def dsigma_dtheta_SM(theta, sqrt_s):
    # Return the differential cross-section (in units of GeV^-2) for SM Bhabha scattering as a function of theta and sqrt_s (given in units of GeV)
    MZ2 = 91.1876**2  # Z boson mass squared in GeV^2
    Alfa = 1/128  # Fine-structure constant
    sw = 0.231**0.5  # sine of the weak mixing angle
    cw = (1 - sw**2)**0.5  # cosine of the weak mixing angle

    cos = np.cos(theta)
    S = sqrt_s**2  # Mandelstam variable S, which is the square of the centre-of-mass energy

    return (Alfa**2*
        (512*(MZ2 - S)**2*sw**4*(cw**2*(4*MZ2 + S - cos*S)- (-1 + cos)*S*sw**2)**2
          + 2*(-1 + cos)**2*(1 + cos)**2*S**2*(-4*MZ2 + S + cos*S)**2*(cw**4 - 2*cw**2*sw**2 - 3*sw**4)**2
          + ((-1 + cos)*cw**4*S*(-4*MZ2 + S + cos*S) + 2*cw**2*(16*MZ2**2 - 8*(1 + cos)*MZ2*S + (-5 + 4*cos + cos**2)*S**2)*sw**2 + (-1 + cos)*S*(-12*MZ2 + (9 + cos)*S)*sw**4)
            * ((-1 + cos)*(1 + cos)**2*cw**4*S*(-4*MZ2 + S + cos*S)
                + 2*cw**2*(16*(1 + 3*cos**2)*MZ2**2- 8*(1 + 3*cos + cos**2 + 3*cos**3)*MZ2*S + (-5 + 2*cos - 12*cos**2 + 14*cos**3 + cos**4)*S**2)*sw**2
                + (-1 + cos)*S*(-4*(3 + 14*cos + 3*cos**2)*MZ2 + (9 + 3*cos + 27*cos**2 + cos**3)*S)*sw**4)
          + ((-1 + cos)*cw**4*S*(-4*MZ2 + S + cos*S) - 2*cw**2*(cos**2*(8*MZ2 - 5*S)*S + S*(8*MZ2 + S) + 4*cos*(-4*MZ2**2 + S**2))*sw**2 + (-1 + cos)*S*(-28*MZ2 + S + 9*cos*S)*sw**4)
            * ((-1 + cos)*(1 + cos)**2*cw**4*S*(-4*MZ2 + S + cos*S) - 2*cw**2*(cos**4*(8*MZ2 - 5*S)*S + 4*cos**2*(8*MZ2 - 3*S)*S + S*(8*MZ2 + S) + 2*cos**3*(-8*MZ2**2 + S**2) + cos*(-48*MZ2**2 + 16*MZ2*S + 14*S**2))*sw**2
               + (-1 + cos)*S*(-4*(7 + 6*cos + 7*cos**2)*MZ2 + (1 + 27*cos + 3*cos**2 + 9*cos**3)*S)*sw**4)
        )) / (1024.*(-1 + cos)**2*cw**4*(MZ2 - S)**2*S*(2*MZ2 + S - cos*S)**2*sw**4)

def dsigma_dtheta_SM_240(theta):
    # Return the differential cross-section (in units of GeV^-2) for SM Bhabha scattering at sqrt_s = 240 GeV
    return dsigma_dtheta_SM(theta, 240)

Make a 2 dimensional array, in the same format as the experimental data file we loaded, i.e.:
$$[\mathtt{x\_low \quad x\_high \quad value \quad error}],$$
but now with theoretical predictions, with errors from the Monte-Carlo function

In [ ]:
theory_predictions = np.zeros_like(experimental_data) # Set up an empty array to hold the predictions, the same shape as the experimental data
for i, row in enumerate(experimental_data):
    theory_predictions[i, 0] = # The same as the experimental bin lower edge
    theory_predictions[i, 1] = # The same as the experimental bin upper edge
    theory_pred, theory_unc = # Use the Monte-Carlo function to calculate the theory prediction and uncertainty
    # Then put these values into the third and fourth columns of the theory_predictions array

#### Exercise 5b: Estimating the luminosity

Now we have two sets of results, the experimental measurements of number of events, and theoretical predictions for the cross-section, we can compare them to estimate the luminosity of the data.
We want to make a new 2 dimensional array, this time containing our estimate of the luminosity.

**HINT**: Rearrange the formula $ N_\mathrm{tot} = L_\mathrm{int} \times \sigma $ to solve for the luminosity, and implement that formula in your code.

**ASK US ABOUT**: how to work out the uncertainty on a ratio

In [ ]:
# Use your function from exercise 3d to make theoretical predictions for each of the experimental bins, as found in the data file loaded above.

luminosity_estimates = np.zeros_like(experimental_data) # Set up an empty array to hold the luminosity estimates

for i, row in enumerate(experimental_data):
    # Extract the lower and upper limits of the bin from the array
    lower_limit = # The same as the experimental bin lower edge
    upper_limit = # The same as the experimental bin upper edge
    luminosity = # Implement the formula for luminosity estimate
    luminosity_error = # Implement the formula for luminosity error as we have discussed with you
    luminosity_estimates[i] = [lower_limit, upper_limit, luminosity, luminosity_error]  # Store the results in the array

___

## Plotting revisited

#### Exercise 5c: Basic data processing

For the purpose of plotting, we more often want data in the form
$$ \mathtt{x\_centre} \quad \mathtt{bin\_width} \quad \mathtt{value} \quad \mathtt{error} $$

Write a function convert our array with the luminosity calculation to the format we want

In [ ]:
import numpy as np

def convert_array(array):
    # Write your code here
    new_array = np.zeros_like(array) # Create a new array with the same shape as the input array
    new_array[:, 0] = # What should go in this column?
    new_array[:, 1] = # What should go in this column?
    new_array[:, 2] = # What should go in this column?
    new_array[:, 3] = # What should go in this column?

print(convert_array(luminosity_estimates))

#### Exercise 5d: Producing the plot

We will now plot our estimate of the integrated luminosity from each different bin of the data from `small-angle-data.dat`.

Remember the guidance from the introduction of what goes into a good plot!

You can use the previous variables and functions you have created/used earlier.

Some useful links for matplotlib:
- [Documentation for errorbar function](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.errorbar.html)
- [Example 1](https://matplotlib.org/stable/gallery/statistics/errorbar.html) and [Example 2](https://matplotlib.org/stable/gallery/statistics/errorbar_features.html) of using the errorbar function

In [ ]:
import matplotlib.pyplot as plt

# Tell matplotlib to use LaTeX rendering, and a large font size
plt.rc('text', usetex=True)
plt.rc('font', family='serif', size=18)

# Plot the ratio of experimental data to theoretical prediction with errors using the plt.errorbar function
converted_luminosity_estimate = convert_array(luminosity_estimates)
# See how we can plot all the data at once
plt.errorbar(converted_luminosity_estimate[0], converted_luminosity_estimate[1], yerr=converted_luminosity_estimate[3], xerr=converted_luminosity_estimate[1])


# Add x and y axis labels

# Now show the plot you have made
plt.show()

Since it looks very consistent, average the five values to produce a combined luminosity value.

In [ ]:
# Calculate the mean values of our luminosity estimates